In [1]:
import pandas as pd
import time

In [15]:
client_id = "q2AL1SGhyj2M1g"
secret = "EGMZoP5NOcaKEqK8SI9Cha65hCoIIw"
user_agent = "CISC351_Webscrape"

import praw
reddit = praw.Reddit(client_id= client_id, client_secret=secret, user_agent=user_agent)

In [16]:
from prawcore.exceptions import Forbidden
test_post = reddit.subreddit('pennystocks').hot(limit=5)
for post in test_post:
    print(post.title)
    print(post.id)

THE LOUNGE: thoughts, noob stuff, low effort, brokers, zero DD, non pennies - February 22, 2021
lpezuc
Daily Plays - February 22, 2021
lpihf3
High Tide ($HITI) Discussions for Week of Feb 22nd
lpl9zp
RXMD - Progressive Care OTCQB - 40 Million Revenue - Undervalued Gem - Due Diligence Package
lpno75
ASRT nasdaq compliance today!
lppplm


First we will need to collect thousands of submissions, to get their ids and their dates. We can then decide on timeframes. 
We also still need to decide on whether to collect them by new, top, or hot. 
I collect three submission ids here for example purposes. In the real version of this, we will collect >10000?

In [124]:
sub_id_date = []
for submission in reddit.subreddit('pennystocks').hot(limit=5):
    sub_id_date.append([submission.id, submission.created, submission.title])
sub_id_date = pd.DataFrame(sub_id_date,columns=['id', 'date', 'title'])
sub_id_date

,id,date,title
0,lpezuc,1.613997e+09,"THE LOUNGE: thoughts, noob stuff, low effort, ..."
1,lpihf3,1.614009e+09,"Daily Plays - February 22, 2021"
2,lptr3i,1.614042e+09,Is anyone else having a second week of red and...
3,lpl9zp,1.614020e+09,High Tide ($HITI) Discussions for Week of Feb ...
4,lppplm,1.614034e+09,ASRT nasdaq compliance today!


Okay as you can see, this will be the future dataset that we can tweak to create new datasets of what submissions we want to use base on the dates.

Okay first lets grab the info from all of these awards. 

In [161]:
gild_info = pd.read_csv("C:/Users/nalab/Desktop/Lectures/CISC351/gilds.csv", header=0)
gild_info = gild_info.set_index("gild")
gild_info

,Name,Description,Value
gild,,,
gid_1,Silver,Shows the Silver Award… and that’s it.,100.0
gid_2,Gold,"Gives the author a week of Reddit Premium, %{c...",500.0
gid_3,Platinum,"Gives the author a month of Reddit Premium, wh...",1800.0
award_5eac457f-ebac-449b-93a7-eb17b557f03c,LOVE!,"When you follow your heart, love is the answer",20.0
award_abb865cf-620b-4219-8777-3658cf9091fb,Starstruck,Can’t stop seeing stars,20.0
...,...,...,...
award_1f0462ee-18f5-4f33-89cf-f1f79336a452,Wholesome (Pro),When you come across a feel-good thing. Gives ...,500.0
award_35c78e6e-507b-4f1d-b3d8-ed43840909a8,Pot o’ Coins,The treasure at the end of the rainbow. Gives ...,1000.0
award_9a123cdb-d26d-4d0c-b7fa-46750b8289fa,Cornucopia,A candy cornucopia of love that gives the auth...,2000.0


In [162]:
for sub in sub_id_date["id"]:
    post = reddit.submission(sub)
    df = pd.DataFrame.from_dict(post.gildings, orient="index",columns=[post])
    gild_info = pd.concat([gild_info, df], axis=1)

gild_info

,Name,Description,Value,lpezuc,lpihf3,lptr3i,lpl9zp,lppplm
gid_1,Silver,Shows the Silver Award… and that’s it.,100.0,1.0,1.0,5.0,5.0,2.0
gid_2,Gold,"Gives the author a week of Reddit Premium, %{c...",500.0,NaN,NaN,NaN,1.0,NaN
gid_3,Platinum,"Gives the author a month of Reddit Premium, wh...",1800.0,NaN,NaN,NaN,NaN,NaN
award_5eac457f-ebac-449b-93a7-eb17b557f03c,LOVE!,"When you follow your heart, love is the answer",20.0,NaN,NaN,NaN,NaN,NaN
award_abb865cf-620b-4219-8777-3658cf9091fb,Starstruck,Can’t stop seeing stars,20.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
award_1f0462ee-18f5-4f33-89cf-f1f79336a452,Wholesome (Pro),When you come across a feel-good thing. Gives ...,500.0,NaN,NaN,NaN,NaN,NaN
award_35c78e6e-507b-4f1d-b3d8-ed43840909a8,Pot o’ Coins,The treasure at the end of the rainbow. Gives ...,1000.0,NaN,NaN,NaN,NaN,NaN
award_9a123cdb-d26d-4d0c-b7fa-46750b8289fa,Cornucopia,A candy cornucopia of love that gives the auth...,2000.0,NaN,NaN,NaN,NaN,NaN
award_4ca5a4e6-8873-4ac5-99b9-71b1d5161a91,Argentium,Latin for distinguished. Shimmers like silver ...,20000.0,NaN,NaN,NaN,NaN,NaN


We can match this dataset to the master set later. 
Now we can grab everything about each submission.

In [154]:
sub_info = [] 

for sub in sub_id_date["id"]:
    post = reddit.submission(sub)
    sub_info.append([post.created_utc,
                     post.distinguished,
                     post.edited,
                     post.id,
                     post.is_original_content,
                     post.is_self, 
                     post.link_flair_template_id, 
                     post.link_flair_text, 
                     post.locked, post.name,
                     post.num_comments, 
                     post.over_18, 
                     post.saved, 
                     post.score, 
                     post.selftext,
                     post.spoiler, 
                     post.title, 
                     post.upvote_ratio, 
                     post.total_awards_received])

#I know i need to add column names in the command below.
#I shall save something so tedious for a future time. 
sub_info = pd.DataFrame(sub_info)
sub_info

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.613968e+09,None,False,lpezuc,False,True,0bbfba84-8b29-11ea-93c0-0e43a0ad59b7,Megathread,False,t3_lpezuc,920,False,False,68,"In an effort to keep the ""main sub"" a little c...",False,"THE LOUNGE: thoughts, noob stuff, low effort, ...",0.95,3
1,1.613981e+09,None,False,lpihf3,False,True,0bbfba84-8b29-11ea-93c0-0e43a0ad59b7,Megathread,False,t3_lpihf3,1464,False,False,130,\n\nTalk about your plays today or things you...,False,"Daily Plays - February 22, 2021",0.95,3
2,1.614014e+09,None,False,lptr3i,False,True,799d62ae-72ab-11ea-88fb-0ecff0e8a853,General Discussion,True,t3_lptr3i,954,False,False,3270,I was really hoping that this week would be a ...,False,Is anyone else having a second week of red and...,0.93,26
3,1.613991e+09,None,False,lpl9zp,False,True,feebbc24-c034-11e3-8d11-12313b08b021,DD :DD:,False,t3_lpl9zp,315,False,False,960,"Hello,\n\nI created [a post on Feb 8th, highli...",False,High Tide ($HITI) Discussions for Week of Feb ...,0.97,24
4,1.614005e+09,None,1.61402e+09,lppplm,False,True,923ead52-ab9c-11ea-84b8-0e2c54eb2d31,Catalyst,False,t3_lppplm,120,False,False,380,$ASRT will complete maintaining their nasdaq c...,False,ASRT nasdaq compliance today!,0.97,6


In [ ]:
Some of the submissions may have deleted authors, so I shall do it separately. 

In [158]:
sub_author_info = []

for sub in sub_info[3]:
    sub_test = reddit.submission(sub)
    if sub_test.author is not None:
        sub_author_info.append([sub_test.author.comment_karma, 
                                sub_test.author.created_utc,
                                sub_test.author.has_verified_email,
                                sub_test.author.id,
                                sub_test.author.name, 
                                sub_test.id])

sub_author_info = pd.DataFrame(sub_author_info, columns = ["sub_author_commentkarma",
                                                                    "sub_author_date",
                                                                    "sub_author_verify_email",
                                                                    "sub_author_id",
                                                                    "sub_author_name",
                                                           "submission_id"])       
sub_author_info

,sub_author_commentkarma,sub_author_date,sub_author_verify_email,sub_author_id,sub_author_name,submission_id
0,180,1.594271e+09,True,77oadqal,PennyBotWeekly,lpezuc
1,180,1.594271e+09,True,77oadqal,PennyBotWeekly,lpihf3
2,461,1.551915e+09,True,3cxrktle,gone_n_done_it,lptr3i
3,14627,1.312771e+09,True,5mtvb,Capriano,lpl9zp
4,307,1.609919e+09,True,9onso05a,QuantopiaNL,lppplm


In [139]:
comment_info = []
def checkComments(comments):
  for comment in comments:
    comment_info.append([comment.body, 
                         comment.created_utc,
                         comment.distinguished,
                         comment.edited,
                         comment.id,
                         comment.is_submitter,
                         comment.parent_id,
                         comment.score,
                         comment.submission.id])
    checkComments(comment.replies)

def processSub(sub):
  sub.comments.replace_more(limit=None,threshold=0)
  checkComments(sub.comments)
    
for sub in sub_id_date["id"]:
    post = reddit.submission(sub)
    processSub(post)

comment_info = pd.DataFrame(comment_info, columns = ["body",
                                                     "date",
                                                     "distinguished",
                                                     "edited",
                                                     "id",
                                                     "is_submitter",
                                                     "parent_id",
                                                     "score",
                                                     "submission_id"])

In [140]:
comment_info

,body,date,distinguished,edited,id,is_submitter,parent_id,score,submission_id
0,Who the heck is buying 3.5 million shares of C...,1.613990e+09,None,False,gobsk5e,False,t3_lpezuc,26,lpezuc
1,Wish I could say it was me lol,1.613995e+09,None,False,gobxu45,False,t1_gobsk5e,6,lpezuc
2,Someone that knows a lot more about something ...,1.613999e+09,None,False,goc4fn1,False,t1_gobsk5e,1,lpezuc
3,ICTY looks like a solid lotto?\n\nCTXR. Are pe...,1.613970e+09,None,False,gob4n5a,False,t3_lpezuc,18,lpezuc
4,CTXR huge room for upward movement even at 2,1.613984e+09,None,False,gobmid9,False,t1_gob4n5a,7,lpezuc
...,...,...,...,...,...,...,...,...,...
2902,Capc up next,1.614010e+09,None,False,gocnbk6,False,t3_lppplm,-1,lppplm
2903,"Idk, I’m skeptical. They are only maintaining ...",1.614017e+09,None,False,god64au,False,t3_lppplm,0,lppplm
2904,You can't lose compliance afterwards for a whi...,1.614018e+09,None,False,god6ld3,True,t1_god64au,4,lppplm
2905,Well if it already has compliance and the risk...,1.614023e+09,None,False,godjn9j,False,t1_god6ld3,1,lppplm


For some of the comments, they have deleted authors, so I have to do this separately. 

In [159]:
comment_author_info = []

for comment in comment_info["id"]:
    com_test = reddit.comment(comment)
    if com_test.author is not None:
        comment_author_info.append([com_test.author.comment_karma, 
                                com_test.author.created_utc,
                                com_test.author.has_verified_email,
                                com_test.author.id,
                                com_test.author.name, 
                                    com_test.id, 
                                    com_test.submission.id])

comment_author_info = pd.DataFrame(comment_author_info, columns = ["author_commentkarma",
                                                                    "author_date",
                                                                    "author_verify_email",
                                                                    "author_id",
                                                                    "author_name", 
                                                                   "comment_id",
                                                                   "submission_id"])       

In [160]:
comment_author_info

,author_commentkarma,author_date,author_verify_email,author_id,author_name,comment_id,submission_id
0,1628,1.605285e+09,False,8v2myq85,belangrijke_muis,gobsk5e,lpezuc
1,638,1.592910e+09,True,60hz7ebs,epsteindidnae,gobxu45,lpezuc
2,30881,1.446503e+09,True,rojt0,TheSilencedScream,goc4fn1,lpezuc
3,45354,1.448165e+09,False,sa53t,-Blast-Tyrant-,gob4n5a,lpezuc
4,144,1.524138e+09,True,17v2lmix,U53RN4M3T4K3N,gobmid9,lpezuc
...,...,...,...,...,...,...,...
2885,20,1.553155e+09,True,3g8xs9mo,chris4xo,gocnbk6,lppplm
2886,185,1.607950e+09,True,9bp68uok,Fancy-Interest2812,god64au,lppplm
2887,307,1.609919e+09,True,9onso05a,QuantopiaNL,god6ld3,lppplm
2888,185,1.607950e+09,True,9bp68uok,Fancy-Interest2812,godjn9j,lppplm


We now have several dataframes: awards, submissions, submission author info, comments, and comment author info. At this point we have all the info we need. It is better to do further processing within each dataframe before combining all of them together to make a master set. You can do that quite simply by matching based either submission or comment ids. 